# Bloom Bite: Hugging Face: 02 zero shot classification

bb-hf-02-zsc.ipynb

Notes
1. I'm using anaconda and created the environment as follows: <br>
 `conda create -n hugs python=3.9 tensorflow transformers pytorch jupyter sentencepiece`
2. Calling pipeline() will download the models needed.  Larger models may take some time to download.  pytorch is an especially large one (over 1 GB), and is needed for the summarization pipeline.  Models are stored in anaconda3/pkgs and are shared by all conda environments so don't need to download multiple times.  These can get big - my pkgs folder is over 20 GB.  Example: <br>
`Downloading pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]`
3. pipeline often returns warnings, typically along the following:
    - `Using a pipeline without specifying a model name and revision in production is not recommended.`
       <br>safe to ignore as these are just illustrative examples.  
    - `No model was supplied, defaulted to...`
       <br>safe to ignore as these are just illustrative examples.
    - ``Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.``
       <br> safe to ignore
    - `Some weights of the model checkpoint at... were not used when initializing...`
       <br> safe to ignore
    - ``ValueError: This tokenizer cannot be instantiated. Please make sure you have `sentencepiece` installed in order to use this tokenizer.``
       <br> errors like this typically occur when a module is missing.  With conda you can add with conda install X.
5. Hugging Face NLP Course link: https://huggingface.co/learn/nlp-course/chapter1/1?fw=pt
6. Hugging Face ModelHub: https://huggingface.co/docs/hub/en/models-the-hub
7. Hugging Face TextClassificationPipeline: https://huggingface.co/docs/transformers/v4.37.2/en/main_classes/pipelines#transformers.TextClassificationPipeline

## Helpers

In [13]:
import time
class timer:
    def __init__(self):
        self.t0 = time.time()
        self.t1 = 0
        self.t = -1
        self.state = "INIT"
    def start(self):
        if (self.state != "INIT") and (self.state != "STOP"):
            print("error - cannot start already started timer")
            return
        self.t0 = time.time()  # restart from now
        self.state = "START"
    def restart(self):
        self.state = "STOP"
        self.start()
    def stopandprint(self):
        self.stop()
        self.print()
    def stop(self):
        if (self.state != "INIT") and (self.state != "START"):
            print("error - cannot stop already stopped timer")
            return
        self.t1 = time.time()
        self.t = self.t1 - self.t0
        self.state = "STOP"
        return self.t
    def print(self):
        if self.state == "STOP":
            print("Elapsed seconds: {:8.3f}".format(self.t))
        else:
            tmp = time.time() - self.t0
            print("Elapsed seconds: {:8.3f}...".format(tmp))

# Previously: Pipeline and sentiment analysis

In [1]:
from transformers import pipeline
classifier = pipeline(task="sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [2]:
classifier("This is one great television show.")

[{'label': 'POSITIVE', 'score': 0.9998749494552612}]

In [3]:
classifier("This is one of the worst movies I have ever seen.")

[{'label': 'NEGATIVE', 'score': 0.9997605681419373}]

# 2. Text Classification with zero shot learning

In [4]:
classifier = pipeline(task="zero-shot-classification")

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [5]:
labels = ["education", "politics", "business"]

In [6]:
R = classifier("This is a course about the Transformers library", labels)

In [7]:
R

{'sequence': 'This is a course about the Transformers library',
 'labels': ['education', 'business', 'politics'],
 'scores': [0.8445982336997986, 0.1119748204946518, 0.043426938354969025]}

In [8]:
print(sum(R["scores"]))

0.9999999925494194


In [9]:
P = []
P.append("the quick brown fox jumped over the lazy dog.")
P.append("man bites dog, film at 11")
P.append("tobacco shown addictive")

In [10]:
#classifier(P, labels)

In [11]:
def classify(textList, candidate_labels):
    t = timer()
    returnList = True
    if type(textList) == type("foo"):
        textList = [textList]
        returnList = False
    printStatus = False
    
    if len(textList) > 2:
        printStatus = True
        lastStatus = 0
        statusInterval = .1

    R = []
    for index, text in enumerate(textList):
        R.append( classifier(text, candidate_labels = candidate_labels))
        if printStatus:
            pct = index / len(P)
            if pct > lastStatus + statusInterval:
                print(".",end="")
                lastStatus = pct
    t.stopandprint()
    if returnList:
        return R
    elif len(R) == 1:
        return R[0]
    else:
        print("error - bad logic")
        return

In [14]:
classify(P, labels)

..Elapsed seconds:   28.571


[{'sequence': 'the quick brown fox jumped over the lazy dog.',
  'labels': ['business', 'education', 'politics'],
  'scores': [0.633482038974762, 0.2199830561876297, 0.14653486013412476]},
 {'sequence': 'man bites dog, film at 11',
  'labels': ['business', 'politics', 'education'],
  'scores': [0.44846415519714355, 0.2853669226169586, 0.26616889238357544]},
 {'sequence': 'tobacco shown addictive',
  'labels': ['business', 'education', 'politics'],
  'scores': [0.3697509765625, 0.3159865140914917, 0.3142625391483307]}]

In [15]:
p = "man bites dog in suprise ice cream heist"
list_of_candidate_labels = []
list_of_candidate_labels.append(["sports", "politics", "news"])
list_of_candidate_labels.append(["sports", "politics", "news", "education"])
list_of_candidate_labels.append(["sports", "politics", "news", "animals"])
list_of_candidate_labels.append(["animals", "sports", "politics", "news"])
list_of_candidate_labels.append(["animals", "sports", "politics", "news", "cats", "penguins", "dogs", "people", "violence"])

In [16]:
listOfR = []
for clabs in list_of_candidate_labels:
    listOfR.append( classify(p, clabs))

Elapsed seconds:   12.618
Elapsed seconds:   16.472
Elapsed seconds:   15.827
Elapsed seconds:   15.307
Elapsed seconds:   38.364


In [17]:
listOfR

[{'sequence': 'man bites dog in suprise ice cream heist',
  'labels': ['news', 'sports', 'politics'],
  'scores': [0.977099597454071, 0.01595023274421692, 0.006950237788259983]},
 {'sequence': 'man bites dog in suprise ice cream heist',
  'labels': ['news', 'sports', 'politics', 'education'],
  'scores': [0.9721549153327942,
   0.015869515016674995,
   0.006915065925568342,
   0.005060562398284674]},
 {'sequence': 'man bites dog in suprise ice cream heist',
  'labels': ['animals', 'news', 'sports', 'politics'],
  'scores': [0.687698483467102,
   0.305149644613266,
   0.0049812812358140945,
   0.0021705697290599346]},
 {'sequence': 'man bites dog in suprise ice cream heist',
  'labels': ['animals', 'news', 'sports', 'politics'],
  'scores': [0.687698483467102,
   0.305149644613266,
   0.0049812812358140945,
   0.0021705697290599346]},
 {'sequence': 'man bites dog in suprise ice cream heist',
  'labels': ['violence',
   'animals',
   'news',
   'dogs',
   'people',
   'sports',
   'polit

In [19]:
for R in listOfR:
    print("labels:", R["labels"])
    for i in range(len(R["labels"])):
        print("  {:15s} {:5.2f}  {:5.2f}".format(R["labels"][i], R["scores"][i], sum(R["scores"][0:i+1])))

labels: ['news', 'sports', 'politics']
  news             0.98   0.98
  sports           0.02   0.99
  politics         0.01   1.00
labels: ['news', 'sports', 'politics', 'education']
  news             0.97   0.97
  sports           0.02   0.99
  politics         0.01   0.99
  education        0.01   1.00
labels: ['animals', 'news', 'sports', 'politics']
  animals          0.69   0.69
  news             0.31   0.99
  sports           0.00   1.00
  politics         0.00   1.00
labels: ['animals', 'news', 'sports', 'politics']
  animals          0.69   0.69
  news             0.31   0.99
  sports           0.00   1.00
  politics         0.00   1.00
labels: ['violence', 'animals', 'news', 'dogs', 'people', 'sports', 'politics', 'cats', 'penguins']
  violence         0.50   0.50
  animals          0.30   0.80
  news             0.13   0.93
  dogs             0.06   0.99
  people           0.00   1.00
  sports           0.00   1.00
  politics         0.00   1.00
  cats             0.00   1

# json data from kaggl

1. Misra, Rishabh. "News Category Dataset." arXiv preprint arXiv:2209.11429 (2022).
2. Misra, Rishabh and Jigyasa Grover. "Sculpting Data for ML: The first act of Machine Learning." ISBN 9798585463570 (2021).

In [20]:
import json
import os
# https://www.kaggle.com/datasets/rmisra/news-category-dataset/data

fn = os.path.expanduser('~') + "/Dropbox/datasets/News_Category_Dataset_v3.json"

with open(fn) as file:
    data = json.load(file)

JSONDecodeError: Extra data: line 2 column 1 (char 448)

In [21]:
datastore = []
for line in open(fn, 'r'):
    datastore.append( json.loads(line) )

In [23]:
len(datastore)

209527

In [24]:
datastore[3:5]

[{'link': 'https://www.huffpost.com/entry/funniest-parenting-tweets_l_632d7d15e4b0d12b5403e479',
  'headline': 'The Funniest Tweets From Parents This Week (Sept. 17-23)',
  'category': 'PARENTING',
  'short_description': '"Accidentally put grown-up toothpaste on my toddler’s toothbrush and he screamed like I was cleaning his teeth with a Carolina Reaper dipped in Tabasco sauce."',
  'authors': 'Caroline Bologna',
  'date': '2022-09-23'},
 {'link': 'https://www.huffpost.com/entry/amy-cooper-loses-discrimination-lawsuit-franklin-templeton_n_632c6463e4b09d8701bd227e',
  'headline': 'Woman Who Called Cops On Black Bird-Watcher Loses Lawsuit Against Ex-Employer',
  'category': 'U.S. NEWS',
  'short_description': 'Amy Cooper accused investment firm Franklin Templeton of unfairly firing her and branding her a racist after video of the Central Park encounter went viral.',
  'authors': 'Nina Golgowski',
  'date': '2022-09-22'}]

In [26]:
categories = {}
for d in datastore:
    cname = d["category"]
    if cname in categories:
        categories[cname] += 1
    else:
        categories[cname] = 1
categories

{'U.S. NEWS': 1377,
 'COMEDY': 5400,
 'PARENTING': 8791,
 'WORLD NEWS': 3299,
 'CULTURE & ARTS': 1074,
 'TECH': 2104,
 'SPORTS': 5077,
 'ENTERTAINMENT': 17362,
 'POLITICS': 35602,
 'WEIRD NEWS': 2777,
 'ENVIRONMENT': 1444,
 'EDUCATION': 1014,
 'CRIME': 3562,
 'SCIENCE': 2206,
 'WELLNESS': 17945,
 'BUSINESS': 5992,
 'STYLE & BEAUTY': 9814,
 'FOOD & DRINK': 6340,
 'MEDIA': 2944,
 'QUEER VOICES': 6347,
 'HOME & LIVING': 4320,
 'WOMEN': 3572,
 'BLACK VOICES': 4583,
 'TRAVEL': 9900,
 'MONEY': 1756,
 'RELIGION': 2577,
 'LATINO VOICES': 1130,
 'IMPACT': 3484,
 'WEDDINGS': 3653,
 'COLLEGE': 1144,
 'PARENTS': 3955,
 'ARTS & CULTURE': 1339,
 'STYLE': 2254,
 'GREEN': 2622,
 'TASTE': 2096,
 'HEALTHY LIVING': 6694,
 'THE WORLDPOST': 3664,
 'GOOD NEWS': 1398,
 'WORLDPOST': 2579,
 'FIFTY': 1401,
 'ARTS': 1509,
 'DIVORCE': 3426}

In [27]:
import pandas as pd
df = pd.DataFrame(datastore)

In [28]:
df

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22
...,...,...,...,...,...,...
209522,https://www.huffingtonpost.com/entry/rim-ceo-t...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,"Reuters, Reuters",2012-01-28
209523,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",,2012-01-28
209524,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",,2012-01-28
209525,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,,2012-01-28


In [30]:
df["category"].unique()

array(['U.S. NEWS', 'COMEDY', 'PARENTING', 'WORLD NEWS', 'CULTURE & ARTS',
       'TECH', 'SPORTS', 'ENTERTAINMENT', 'POLITICS', 'WEIRD NEWS',
       'ENVIRONMENT', 'EDUCATION', 'CRIME', 'SCIENCE', 'WELLNESS',
       'BUSINESS', 'STYLE & BEAUTY', 'FOOD & DRINK', 'MEDIA',
       'QUEER VOICES', 'HOME & LIVING', 'WOMEN', 'BLACK VOICES', 'TRAVEL',
       'MONEY', 'RELIGION', 'LATINO VOICES', 'IMPACT', 'WEDDINGS',
       'COLLEGE', 'PARENTS', 'ARTS & CULTURE', 'STYLE', 'GREEN', 'TASTE',
       'HEALTHY LIVING', 'THE WORLDPOST', 'GOOD NEWS', 'WORLDPOST',
       'FIFTY', 'ARTS', 'DIVORCE'], dtype=object)

In [31]:
len(df["category"].unique())

42

In [33]:
ourcats = ['CRIME', 'TECH', 'MONEY', 'WORLD NEWS', 'EDUCATION']

In [34]:
df2 = df[df["category"].isin( ourcats )]

In [35]:
df.shape

(209527, 6)

In [36]:
df2.shape

(11735, 6)

In [38]:
df2["category"].unique()

array(['WORLD NEWS', 'TECH', 'EDUCATION', 'CRIME', 'MONEY'], dtype=object)

In [45]:
df3 = df2.sample(10)

In [46]:
df3

,link,headline,category,short_description,authors,date
195552,https://www.huffingtonpost.com/entry/top-6-way...,Top 6 Ways to Pay Off Any Loan Faster,MONEY,"After paying off all my credit cards, my auto ...","Carrie Smith, Contributor\nFinancial Writer & ...",2012-06-26
203280,https://www.huffingtonpost.com/entry/marlinde-...,"Mirlande Wilson, Alleged Mega Millions Winner,...",MONEY,"But when pressed for details by reporters, Wil...",,2012-04-04
26324,https://www.huffingtonpost.com/entry/why-us-al...,Why U.S. Allies In The Middle East Should Be A...,WORLD NEWS,North Korea’s latest ballistic missile launche...,"Ephraim Sneh, Contributorformer Israeli Deputy...",2017-08-30
182820,https://www.huffingtonpost.com/entry/china-cop...,China's Copyright Piracy Problem Isn't As Big ...,TECH,"By Ben Blanchard ""This could only happen becau...","Reuters, Reuters",2012-11-11
30184,https://www.huffingtonpost.com/entry/do-girls-...,Do Girls Have Access To Team Sports? Many Cali...,EDUCATION,No research has found that girls are less inte...,"EdSource, ContributorNonprofit news site focus...",2017-07-14
170974,https://www.huffingtonpost.com/entry/steubenvi...,Steubenville Rape Trial Verdict To Be Handed D...,CRIME,Defense attorney Walter Madison countered by h...,"Reuters, Reuters",2013-03-17
90745,https://www.huffingtonpost.com/entry/hackers-a...,Law Enforcement To Dark Web Hackers: Give Us D...,TECH,"""We [are] appealing to you to do the right thi...",Dana Liebelson,2015-08-24
9639,https://www.huffingtonpost.com/entry/george-zi...,George Zimmerman Accused Of Stalking Detective...,CRIME,Zimmerman appeared to threaten to feed the inv...,David Moye,2018-05-08
91769,https://www.huffingtonpost.com/entry/beauty-qu...,Beauty Queen Arrested For Allegedly Faking Can...,CRIME,"One fundraising event brought in $14,000, auth...",Simon McCormack,2015-08-12
125757,https://www.huffingtonpost.com/entry/how-to-pr...,How to Prevent Summertime Scams,CRIME,Summertime is here -- and that means sunshine ...,"Robert Siciliano, ContributorPersonal Security...",2014-07-19


In [47]:
P = list(df3["short_description"])

In [48]:
R = classify(P, ourcats)

....Elapsed seconds:   62.128


In [49]:
R

[{'sequence': 'After paying off all my credit cards, my auto loan was the only thing between me and complete debt freedom. Here are some of the best methods to pay off your car loan, credit cards, or any type of debt even faster.',
  'labels': ['TECH', 'MONEY', 'WORLD NEWS', 'EDUCATION', 'CRIME'],
  'scores': [0.3999278247356415,
   0.38003939390182495,
   0.09095016866922379,
   0.07218597829341888,
   0.05689667537808418]},
 {'sequence': 'But when pressed for details by reporters, Wilson changed her story and said that a coworker purchased the tickets on her',
  'labels': ['MONEY', 'TECH', 'CRIME', 'WORLD NEWS', 'EDUCATION'],
  'scores': [0.42913618683815,
   0.284102201461792,
   0.12919659912586212,
   0.09391582757234573,
   0.06364918500185013]},
 {'sequence': 'North Korea’s latest ballistic missile launches remind us that this remains a crisis the Trump administration is forced to',
  'labels': ['TECH', 'CRIME', 'WORLD NEWS', 'EDUCATION', 'MONEY'],
  'scores': [0.335333645343780

In [63]:
for i, r in enumerate(R):
    print("{:2d} {:}".format(i+1, r['sequence']))
    for j in range(2):
        print("   {:15s}  {:5.3f}".format(r["labels"][j], r["scores"][j]))
    print(  "   actual: ", df3.iloc[i]["category"])
    print(df3.iloc[i]["short_description"][0:40])


 1 After paying off all my credit cards, my auto loan was the only thing between me and complete debt freedom. Here are some of the best methods to pay off your car loan, credit cards, or any type of debt even faster.
   TECH             0.400
   MONEY            0.380
   actual:  MONEY
After paying off all my credit cards, my
 2 But when pressed for details by reporters, Wilson changed her story and said that a coworker purchased the tickets on her
   MONEY            0.429
   TECH             0.284
   actual:  MONEY
But when pressed for details by reporter
 3 North Korea’s latest ballistic missile launches remind us that this remains a crisis the Trump administration is forced to
   TECH             0.335
   CRIME            0.298
   actual:  WORLD NEWS
North Korea’s latest ballistic missile l
 4 By Ben Blanchard "This could only happen because China's intellectual property rights environment sets foreign investors
   TECH             0.350
   WORLD NEWS       0.258
   actual:  TECH


In [68]:
P1 = list(df3["short_description"])
P2 = list(df3["headline"])
P3 = []
for i in range(len(P1)):
    P3.append(P2[i] + ": " + P1[i])
    

In [69]:
P3

['Top 6 Ways to Pay Off Any Loan Faster: After paying off all my credit cards, my auto loan was the only thing between me and complete debt freedom. Here are some of the best methods to pay off your car loan, credit cards, or any type of debt even faster.',
 "Mirlande Wilson, Alleged Mega Millions Winner, Claims Ticket Is Hidden In McDonald's: But when pressed for details by reporters, Wilson changed her story and said that a coworker purchased the tickets on her",
 'Why U.S. Allies In The Middle East Should Be Alarmed By North Korea: North Korea’s latest ballistic missile launches remind us that this remains a crisis the Trump administration is forced to',
 'China\'s Copyright Piracy Problem Isn\'t As Big As Media Makes It, Says China: By Ben Blanchard "This could only happen because China\'s intellectual property rights environment sets foreign investors',
 "Do Girls Have Access To Team Sports? Many California Schools Aren't Telling, Despite Law: No research has found that girls are 

In [70]:
R2 = classify(P2, ourcats)

....Elapsed seconds:   99.657


In [73]:
for i, r in enumerate(R2):
    print("{:2d} {:}".format(i+1, r['sequence']))
    for j in range(2):
        print("   {:15s}  {:5.3f}".format(r["labels"][j], r["scores"][j]))
    print(  "   actual: ", df3.iloc[i]["category"])
    #print(df3.iloc[i]["short_description"][0:40])

 1 Top 6 Ways to Pay Off Any Loan Faster
   MONEY            0.402
   TECH             0.355
   actual:  MONEY
 2 Mirlande Wilson, Alleged Mega Millions Winner, Claims Ticket Is Hidden In McDonald's
   MONEY            0.594
   TECH             0.141
   actual:  MONEY
 3 Why U.S. Allies In The Middle East Should Be Alarmed By North Korea
   TECH             0.366
   WORLD NEWS       0.284
   actual:  WORLD NEWS
 4 China's Copyright Piracy Problem Isn't As Big As Media Makes It, Says China
   CRIME            0.384
   TECH             0.248
   actual:  TECH
 5 Do Girls Have Access To Team Sports? Many California Schools Aren't Telling, Despite Law
   EDUCATION        0.298
   TECH             0.272
   actual:  EDUCATION
 6 Steubenville Rape Trial Verdict To Be Handed Down Today
   CRIME            0.833
   WORLD NEWS       0.096
   actual:  CRIME
 7 Law Enforcement To Dark Web Hackers: Give Us Dirt On Ashley Madison Thieves
   CRIME            0.662
   TECH             0.213
   actual: 

In [74]:
R3 = classify(P3, ourcats)

....Elapsed seconds:  122.072


In [75]:
for i, r in enumerate(R3):
    print("{:2d} {:}".format(i+1, r['sequence']))
    for j in range(2):
        print("   {:15s}  {:5.3f}".format(r["labels"][j], r["scores"][j]))
    print(  "   actual: ", df3.iloc[i]["category"])
    #print(df3.iloc[i]["short_description"][0:40])

 1 Top 6 Ways to Pay Off Any Loan Faster: After paying off all my credit cards, my auto loan was the only thing between me and complete debt freedom. Here are some of the best methods to pay off your car loan, credit cards, or any type of debt even faster.
   MONEY            0.412
   TECH             0.388
   actual:  MONEY
 2 Mirlande Wilson, Alleged Mega Millions Winner, Claims Ticket Is Hidden In McDonald's: But when pressed for details by reporters, Wilson changed her story and said that a coworker purchased the tickets on her
   MONEY            0.562
   TECH             0.221
   actual:  MONEY
 3 Why U.S. Allies In The Middle East Should Be Alarmed By North Korea: North Korea’s latest ballistic missile launches remind us that this remains a crisis the Trump administration is forced to
   TECH             0.438
   WORLD NEWS       0.204
   actual:  WORLD NEWS
 4 China's Copyright Piracy Problem Isn't As Big As Media Makes It, Says China: By Ben Blanchard "This could only happen b